In [ ]:
# insecure_app.py - intentionally insecure example
import os
import sqlite3
import subprocess
import pickle
import logging
from werkzeug.utils import secure_filename
logging.basicConfig(level=logging.DEBUG)

DB = "app.db"

def init_db():
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, username TEXT, password TEXT)")
    conn.commit()
    conn.close()

def register():
    username = input("Choose username: ")
    password = input("Choose password: ")
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    # insecure: storing plain password
    c.execute("INSERT INTO users (username, password) VALUES ('%s', '%s')" % (username, password))
    conn.commit()
    conn.close()
    print("Registered")

# safe_register.py (showing parameterized insert)
import sqlite3

def register_safe(username, password):
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    # secure: use placeholders (?) so DB driver binds parameters safely
    c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
    conn.commit()
    conn.close()


def login():
    username = input("Username: ")
    password = input("Password: ")
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    # insecure: building SQL with string formatting
    c.execute("SELECT id FROM users WHERE username = '%s' AND password = '%s'" % (username, password))
    row = c.fetchone()
    if row:
        print("Login success, user id:", row[0])
        return row[0]
    else:
        print("Login failed")
        return None

def upload_file(user_id):
    filename = input("Enter filename to upload: ")
    filename = secure_filename(filename)
    # insecure: saving file using provided filename without validation
    content = input("Paste file content: ")
    path = "/var/www/uploads/" + filename
    with open(path, "w") as f:
        f.write(content)
    print("Saved", path)

def run_report(user_id):
    cmd = input("Enter report filter: ")
    # insecure: calling shell with user input
    subprocess.run("grep '%s' /var/log/app.log | head -n 10" % cmd, shell=True)

def save_state(obj, name):
    # insecure: using pickle to serialize user objects
    with open(f"/tmp/{name}.pkl", "wb") as f:
        pickle.dump(obj, f)

def load_state(name):
    with open(f"/tmp/{name}.pkl", "rb") as f:
        return pickle.load(f)

def main():
    init_db()
    logging.debug("Starting app")
    while True:
        print("1 register 2 login 3 exit")
        choice = input("> ")
        if choice == "1":
            register()
        elif choice == "2":
            uid = login()
            if uid:
                print("a upload b report c save d load q quit")
                cmd = input("> ")
                if cmd == "a":
                    upload_file(uid)
                elif cmd == "b":
                    run_report(uid)
                elif cmd == "c":
                    save_state({"id": uid}, "state")
                elif cmd == "d":
                    print(load_state("state"))
                elif cmd == "q":
                    break
        elif choice == "3":
            break

if __name__ == "__main__":
    main()
